In [57]:
import pyodbc
import os
import pandas as pd
import numpy as np
import datetime
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
import getpass
import plotly
import plotly.express as px
import plotly.io as pio

import matplotlib.pyplot as plt
%matplotlib inline

start = dt.now()
print(start)

2024-08-05 09:57:38.551816


In [58]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=sql-2-db;'
                      'Database=CBQ2;')
                    
cursor = conn.cursor()

In [59]:
username = getpass.getuser()

prior_day_str = input(f'Good morning {username}, \
please type in the prior day using the following format: yyyy-mm-dd. ')

Good morning RBH, please type in the prior day using the following format: yyyy-mm-dd. 2024-08-02


In [60]:
''' 
Getting some variables for time related processes.
'''  

prior_day_dt = dt.strptime(prior_day_str,'%Y-%m-%d')
prior_day = prior_day_dt.isoformat()
prior_day_ly = prior_day_dt - relativedelta(years=1)
tp = prior_day_dt.strftime('%Y%m')
tply = prior_day_ly.strftime('%Y%m')

In [62]:
# Testing to make ebs.sales file is prior day's file and not from a previous day.

query0 = f'''
Select max(t.LastUpdated) [updated] from metrics.TableLastUpdated t where TableName = 'ebs.sales'
'''

In [63]:
df_q0 = pd.read_sql_query(query0,conn)

In [65]:
last_updated = df_q0.loc[0,'updated']
last_updated = last_updated.to_pydatetime()
last_updated_fmt = last_updated.strftime('%A, %B %d, %Y at %H:%M')

right_now = datetime.datetime.now()
delta = right_now - last_updated
print(delta.days)
print(right_now)
print(last_updated_fmt)
   

2
2024-08-05 09:58:49.292595
Saturday, August 03, 2024 at 04:48


In [66]:
# Prior Day and MTD sales by publishing groups broken up by core and non-core
query1 = f'''SELECT
    case
        when i.PUBLISHER_CODE = 'Chronicle' then 'CB'
        else 'DP'
    END CBDP
    ,case
        when i.PRODUCT_TYPE  in('RP','CP') then 'CPRP'
        when i.PUBLISHING_GROUP in('LIF','FWN') then 'FLS'
        when i.PUBLISHING_GROUP in('ART','CHL','ENT','MOD') then i.PUBLISHING_GROUP
        when i.PUBLISHING_GROUP in('BAR-ART','BAR-ENT','BAR-LIF') then 'BAR'
        when i.PUBLISHING_GROUP = 'DOB' then 'DO'
        when i.PUBLISHER_CODE = 'Tourbillon' then 'TW'
        when i.PUBLISHER_CODE = 'Sierra Club' then 'SC'
        when i.PUBLISHER_CODE = 'Levine Querido' then 'LQ'
        when i.PUBLISHER_CODE IN('Glam Media','Benefit','PQ Blackwell','San Francisco Art Institute','AFO LLC','FareArts','Sager') then 'CD'        when i.PUBLISHER_CODE = 'Creative Company' then 'CC'            
        WHEN i.PUBLISHER_CODE = 'GALISON' AND stie.DISTRIBUTION_DIRECT = 'Y' then 'GAD'  
        when i.PUBLISHING_GROUP = 'GAL' then 'GA'   
        when i.PUBLISHING_GROUP = 'GAL-CL' then 'CL'       
        when i.PUBLISHING_GROUP = 'MUD' then 'MP'
        when i.PUBLISHING_GROUP in('LAU-BIS') then 'LKBS'          
        when i.PUBLISHER_CODE = 'Laurence King' and i.PRODUCT_TYPE = 'FT' then 'LKGI'           
        when i.PUBLISHER_CODE = 'Laurence King' and i.PRODUCT_TYPE <> 'FT' then 'LKBK' 
        when i.PUBLISHER_CODE = 'PRINCETON' and i.PUBLISHING_GROUP in('PAP-MS','PAP-MP') then 'PAMS' 
        when i.PUBLISHER_CODE = 'Princeton' and i.PRODUCT_TYPE = 'FT' then 'PAGI'          
        when i.PUBLISHER_CODE = 'Princeton' and i.PRODUCT_TYPE <> 'FT' then 'PABK' 
        when i.PUBLISHING_GROUP = 'MOL-PLANNER' then 'MSPL'      
        when i.PUBLISHING_GROUP in('MOL-NON PAPER','MOL-PAPER') then 'MSCO' 
        when i.PUBLISHER_CODE = 'Hardie Grant Publishing' then 'HG'
        else i.PUBLISHING_GROUP        
    end tab
    ,sum(case when i.PRODUCT_TYPE in('BK','FT') AND TRX_DATE = {prior_day} then sd.REVENUE_AMOUNT else 0 end) [PriorDay_Core]
    ,sum(case when i.PRODUCT_TYPE in('BK','FT') AND sd.period = {tp} then sd.REVENUE_AMOUNT else 0 end) tp_core
    ,sum(case when i.PRODUCT_TYPE in('BK','FT') AND sd.period = {tply} then sd.REVENUE_AMOUNT else 0 end) tply_core
    ,sum(case when i.PRODUCT_TYPE not in('BK','FT') AND TRX_DATE = ? then sd.REVENUE_AMOUNT else 0 end) [PriorDay_NonCore]
    ,sum(case when i.PRODUCT_TYPE not in('BK','FT') AND sd.period = {tp} then sd.REVENUE_AMOUNT else 0 end) tp_NonCore
    ,sum(case when i.PRODUCT_TYPE not in('BK','FT') AND sd.period = {tply} then sd.REVENUE_AMOUNT else 0 end) tply_NonCore
FROM
    ebs.Sales sd
    inner join ssr.SalesSSRRow stie on stie.CUSTOMER_TRX_LINE_ID = sd.CUSTOMER_TRX_LINE_ID
    INNER JOIN ebs.Item i on i.ITEM_ID = sd.ITEM_ID
WHERE 
    Sd.PERIOD in({tply},{tp})
    AND sd.INVOICE_LINE_TYPE='SALE'
    AND cbq2.dbo.fnSaleTypeCode(SD.AR_TRX_TYPE_ID)='N'
    AND i.PUBLISHER_CODE not in('ZZZ','MKT')
GROUP BY
    case
        when i.PUBLISHER_CODE = 'Chronicle' then 'CB'
        else 'DP'
    END
    ,case
        when i.PRODUCT_TYPE  in('RP','CP') then 'CPRP'
        when i.PUBLISHING_GROUP in('LIF','FWN') then 'FLS'
        when i.PUBLISHING_GROUP in('ART','CHL','ENT','MOD') then i.PUBLISHING_GROUP
        when i.PUBLISHING_GROUP in('BAR-ART','BAR-ENT','BAR-LIF') then 'BAR'
        when i.PUBLISHING_GROUP = 'DOB' then 'DO'
        when i.PUBLISHER_CODE = 'Tourbillon' then 'TW'
        when i.PUBLISHER_CODE = 'Sierra Club' then 'SC'
        when i.PUBLISHER_CODE = 'Levine Querido' then 'LQ'
        when i.PUBLISHER_CODE IN('Glam Media','Benefit','PQ Blackwell','San Francisco Art Institute','AFO LLC','FareArts','Sager') then 'CD'
        when i.PUBLISHER_CODE = 'Creative Company' then 'CC'
        WHEN i.PUBLISHER_CODE = 'GALISON' AND stie.DISTRIBUTION_DIRECT = 'Y' then 'GAD'
        when i.PUBLISHING_GROUP = 'GAL' then 'GA'
        when i.PUBLISHING_GROUP = 'GAL-CL' then 'CL' 
        when i.PUBLISHING_GROUP = 'MUD' then 'MP'
        when i.PUBLISHING_GROUP in('LAU-BIS') then 'LKBS'
        when i.PUBLISHER_CODE = 'Laurence King' and i.PRODUCT_TYPE = 'FT' then 'LKGI'
        when i.PUBLISHER_CODE = 'Laurence King' and i.PRODUCT_TYPE <> 'FT' then 'LKBK'
        when i.PUBLISHER_CODE = 'PRINCETON' and i.PUBLISHING_GROUP in('PAP-MS','PAP-MP') then 'PAMS'
        when i.PUBLISHER_CODE = 'Princeton' and i.PRODUCT_TYPE = 'FT' then 'PAGI' 
        when i.PUBLISHER_CODE = 'Princeton' and i.PRODUCT_TYPE <> 'FT' then 'PABK' 
        when i.PUBLISHING_GROUP = 'MOL-PLANNER' then 'MSPL' 
        when i.PUBLISHING_GROUP in('MOL-NON PAPER','MOL-PAPER') then 'MSCO' 
        when i.PUBLISHER_CODE = 'Hardie Grant Publishing' then 'HG'
        else i.PUBLISHING_GROUP
    end

'''

In [67]:
prior_day

'2024-08-02T00:00:00'

In [68]:
df_q1 = pd.read_sql_query(query1,conn,params=[prior_day])

DatabaseError: Execution failed on sql 'SELECT
    case
        when i.PUBLISHER_CODE = 'Chronicle' then 'CB'
        else 'DP'
    END CBDP
    ,case
        when i.PRODUCT_TYPE  in('RP','CP') then 'CPRP'
        when i.PUBLISHING_GROUP in('LIF','FWN') then 'FLS'
        when i.PUBLISHING_GROUP in('ART','CHL','ENT','MOD') then i.PUBLISHING_GROUP
        when i.PUBLISHING_GROUP in('BAR-ART','BAR-ENT','BAR-LIF') then 'BAR'
        when i.PUBLISHING_GROUP = 'DOB' then 'DO'
        when i.PUBLISHER_CODE = 'Tourbillon' then 'TW'
        when i.PUBLISHER_CODE = 'Sierra Club' then 'SC'
        when i.PUBLISHER_CODE = 'Levine Querido' then 'LQ'
        when i.PUBLISHER_CODE IN('Glam Media','Benefit','PQ Blackwell','San Francisco Art Institute','AFO LLC','FareArts','Sager') then 'CD'        when i.PUBLISHER_CODE = 'Creative Company' then 'CC'            
        WHEN i.PUBLISHER_CODE = 'GALISON' AND stie.DISTRIBUTION_DIRECT = 'Y' then 'GAD'  
        when i.PUBLISHING_GROUP = 'GAL' then 'GA'  

In [30]:
#Top 10 SSR Accounts for prior day
query2 = f'''SELECT TOP 10
    ssr_row.Description ssr_row
    ,sum(case when t.PUBLISHER_CODE = 'Chronicle' then sd.REVENUE_AMOUNT else 0 end) [Chronicle]
    ,sum(case when not(t.PUBLISHER_CODE = 'Chronicle') then sd.REVENUE_AMOUNT else 0 end) [Distribution]
    ,sum(sd.revenue_amount) [Total]
FROM
    ebs.Sales sd
    inner join ssr.SalesSSRRow stie on stie.CUSTOMER_TRX_LINE_ID = sd.CUSTOMER_TRX_LINE_ID
    inner join ssr.SSRRow ssr_row on ssr_row.SSRRowID= stie.SSRRowID
    INNER JOIN ebs.Item t on t.ITEM_ID=sd.ITEM_ID
    INNER JOIN ssr.SubChannel subchan on subchan.SubChannelID = ssr_row.SubChannelID
    inner join ssr.Channel chan on chan.ChannelID = subchan.ChannelID

WHERE 
    Sd.PERIOD = {tp}
    AND TRX_DATE = ?
    AND sd.INVOICE_LINE_TYPE='SALE'
    AND cbq2.dbo.fnSaleTypeCode(SD.AR_TRX_TYPE_ID)='N'
    and t.PRODUCT_TYPE in('BK','FT')
GROUP BY
    ssr_row.Description
ORDER BY
    sum(sd.REVENUE_AMOUNT) desc'''


In [31]:
# Top prior day top title sales by CB and DP
query3 = f'''SELECT
    case
        when t.PUBLISHER_CODE = 'Chronicle' then 'cb'
        else 'dp'
    end div
    ,t.ISBN
    ,t.SHORT_TITLE [Title]
    ,t.PRICE_AMOUNT [Price]
    ,case
        when t.PUBLISHING_GROUP in('LIF','FWN') then 'FLS'
        else t.PUBLISHING_GROUP
    end pgc
    ,sum(sd.QUANTITY_INVOICED) [Units]
    ,sum(sd.REVENUE_AMOUNT) [Dollars]
    ,ROW_NUMBER() over(PARTITION BY case when t.PUBLISHER_CODE = 'Chronicle' then 'cb' else 'dp' end ORDER BY sum(sd.REVENUE_AMOUNT) desc) 'order'

FROM
    ebs.Sales sd
    INNER JOIN ebs.Item t on t.ITEM_ID=sd.ITEM_ID
    INNER JOIN ebs.Customer c on c.SITE_USE_ID=sd.SHIP_TO_SITE_USE_ID
    INNER JOIN ebs.SalesRep sr on sr.SALESREP_ID=sd.PRIMARY_SALESREP_ID
    LEFT OUTER JOIN ebs.Customer BillTo ON BillTo.SITE_USE_ID = c.BILL_TO_SITE_USE_ID
WHERE
    Sd.PERIOD = {tp}
    AND sd.TRX_DATE= ?
    AND sd.INVOICE_LINE_TYPE='SALE'
    AND cbq2.dbo.fnSaleTypeCode(SD.AR_TRX_TYPE_ID)='N'
    AND t.PRICE_AMOUNT <> 0
    AND sr.SALESREP_NUMBER not in ('1055','1061','2055')
    AND not(t.FORMAT in ('EB', 'EN', 'PP', 'PS', 'SA', 'AP'))
    AND not(t.PRODUCT_TYPE in ('DI','PZ','?'))

GROUP BY 
    case
        when t.PUBLISHER_CODE = 'Chronicle' then 'cb'
        else 'dp'
    end
    ,t.ISBN
    ,t.SHORT_TITLE
    ,t.PRICE_AMOUNT
    ,case
        when t.PUBLISHING_GROUP in('LIF','FWN') then 'FLS'
        else t.PUBLISHING_GROUP
    end'''

In [32]:
# Top 10 MTD titles by CB and DP
query4 = f'''SELECT
    case
        when t.PUBLISHER_CODE = 'Chronicle' then 'cb'
        else 'dp'
    end div
    ,t.ISBN
    ,t.SHORT_TITLE [Title]
    ,t.PRICE_AMOUNT [Price]
    ,case
        when t.PUBLISHING_GROUP in('LIF','FWN') then 'FLS'
        else t.PUBLISHING_GROUP
    end pgc
    ,sum(sd.QUANTITY_INVOICED) [Units]
    ,sum(sd.REVENUE_AMOUNT) [Dollars]
    ,ROW_NUMBER() over(PARTITION BY case when t.PUBLISHER_CODE = 'Chronicle' then 'cb' else 'dp' end ORDER BY sum(sd.REVENUE_AMOUNT) desc) 'order'

FROM
    ebs.Sales sd
    INNER JOIN ebs.Item t on t.ITEM_ID=sd.ITEM_ID
    INNER JOIN ebs.Customer c on c.SITE_USE_ID=sd.SHIP_TO_SITE_USE_ID
    INNER JOIN ebs.SalesRep sr on sr.SALESREP_ID=sd.PRIMARY_SALESREP_ID
    LEFT OUTER JOIN ebs.Customer BillTo ON BillTo.SITE_USE_ID = c.BILL_TO_SITE_USE_ID
WHERE
    Sd.PERIOD = {tp}
    AND sd.INVOICE_LINE_TYPE='SALE'
    AND cbq2.dbo.fnSaleTypeCode(SD.AR_TRX_TYPE_ID)='N'
    AND t.PRICE_AMOUNT <> 0
    AND sr.SALESREP_NUMBER not in ('1055','1061','2055')
    AND not(t.FORMAT in ('EB', 'EN', 'PP', 'PS', 'SA', 'AP'))
    AND not(t.PRODUCT_TYPE in ('DI','PZ','?'))

GROUP BY 
    case
        when t.PUBLISHER_CODE = 'Chronicle' then 'cb'
        else 'dp'
    end
    ,t.ISBN
    ,t.SHORT_TITLE
    ,t.PRICE_AMOUNT
    ,case
        when t.PUBLISHING_GROUP in('LIF','FWN') then 'FLS'
        else t.PUBLISHING_GROUP
    end'''

In [33]:
#Napkin Top Forecast query

query5 = f'''SELECT
    case
        when i.PUBLISHING_GROUP in('BAR-ART','BAR-ENT','BAR-FWN','BAR-LIF') then 'BAR'
        when i.PUBLISHING_GROUP in('FWN','LIF') then 'FLS'
        else i.PUBLISHING_GROUP
    end pgrp
    ,sum(case when SSR_row.SSRRowID in('32','146') then sd.REVENUE_AMOUNT else 0 end) [cons]
    ,sum(case when SSR_row.SSRRowID in('6') then sd.REVENUE_AMOUNT else 0 end) [amaz]

FROM
    ebs.Sales sd
    inner join ssr.SalesSSRRow stie on stie.CUSTOMER_TRX_LINE_ID = sd.CUSTOMER_TRX_LINE_ID
        inner join ssr.SSRRow ssr_row on ssr_row.SSRRowID= stie.SSRRowID
        INNER JOIN ebs.Item i on i.ITEM_ID = sd.ITEM_ID

    WHERE
        Sd.PERIOD = {tp}
        AND sd.INVOICE_LINE_TYPE='SALE'
        AND cbq2.dbo.fnSaleTypeCode(SD.AR_TRX_TYPE_ID)='N'
        AND i.PUBLISHING_GROUP not in('ZZZ','MKT')
        and i.PUBLISHER_CODE = 'Chronicle'
        AND SSR_row.SSRRowID in('32','146','6')
    GROUP BY
        case
            when i.PUBLISHING_GROUP in('BAR-ART','BAR-ENT','BAR-FWN','BAR-LIF') then 'BAR'
            when i.PUBLISHING_GROUP in('FWN','LIF') then 'FLS'
            else i.PUBLISHING_GROUP
        end
    ORDER BY
        pgrp'''

In [34]:
# Order Query - Top 10 CB Prior Day SSR_ROW Orders

query6 = f'''SELECT top 10                             
    chan.Description channel
    ,ssr_row.Description SSR_Row
    ,sum(ho.Quantity) qty
FROM                                           
    hachette.HachetteOrders ho
        inner join ebs.item i on i.ITEM_TITLE = ho.isbn     
        inner join ssr.SSRRow ssr_row on ssr_row.SSRRowID = ho.SSRRowID
        inner join ssr.SubChannel subchan on subchan.SubChannelID = ssr_row.SubChannelID
        inner join ssr.Channel chan on chan.ChannelID = subchan.ChannelID                
WHERE                        
    i.PRICE_AMOUNT > 0
    and ho.EnteredDate = ? 
    and i.PUBLISHER_CODE = 'Chronicle'
    GROUP BY                                             
    chan.Description
    ,ssr_row.Description
ORDER BY
    sum(ho.Quantity) desc
'''

In [35]:
# Order Query - Top 10 DP Prior Day SSR_ROW Orders

query7 = f'''SELECT top 10                             
    chan.Description channel
    ,ssr_row.Description SSR_Row
    ,sum(ho.Quantity) qty
FROM                                           
    hachette.HachetteOrders ho
        inner join ebs.item i on i.ITEM_TITLE = ho.isbn     
        inner join ssr.SSRRow ssr_row on ssr_row.SSRRowID = ho.SSRRowID
        inner join ssr.SubChannel subchan on subchan.SubChannelID = ssr_row.SubChannelID
        inner join ssr.Channel chan on chan.ChannelID = subchan.ChannelID                
WHERE                        
    i.PRICE_AMOUNT > 0
    and ho.EnteredDate = ?
    and i.PUBLISHER_CODE != 'Chronicle'
    GROUP BY                                             
    chan.Description
    ,ssr_row.Description
ORDER BY
    sum(ho.Quantity) desc
'''

In [36]:
# Order Query - Top 10 CB Prior Day Title Orders

query8 = f'''SELECT top 10                          
    i.PUBLISHING_GROUP pgrp
    ,i.ISBN
    ,i.SHORT_TITLE
    ,sum(ho.Quantity) qty
FROM                                           
    hachette.HachetteOrders ho
        inner join ebs.item i on i.ITEM_TITLE = ho.isbn     
        inner join ssr.SSRRow ssr_row on ssr_row.SSRRowID = ho.SSRRowID
        inner join ssr.SubChannel subchan on subchan.SubChannelID = ssr_row.SubChannelID
        inner join ssr.Channel chan on chan.ChannelID = subchan.ChannelID                
WHERE                        
    i.PRICE_AMOUNT > 0
    AND i.PUBLISHER_CODE = 'Chronicle'
    and ho.EnteredDate = ?

GROUP BY                                             
    i.PUBLISHING_GROUP
    ,i.ISBN
    ,i.SHORT_TITLE
    ORDER BY
    sum(ho.Quantity) desc

'''

In [37]:
# Order Query - Top 10 CB Prior Day Title Orders

query9 = f'''SELECT top 10                          
    i.PUBLISHER_CODE pub
    ,i.ISBN
    ,i.SHORT_TITLE
    ,sum(ho.Quantity) qty
FROM                                           
    hachette.HachetteOrders ho
        inner join ebs.item i on i.ITEM_TITLE = ho.isbn     
        inner join ssr.SSRRow ssr_row on ssr_row.SSRRowID = ho.SSRRowID
        inner join ssr.SubChannel subchan on subchan.SubChannelID = ssr_row.SubChannelID
        inner join ssr.Channel chan on chan.ChannelID = subchan.ChannelID                
WHERE                        
    i.PRICE_AMOUNT > 0
    AND i.PUBLISHER_CODE != 'Chronicle'
    and ho.EnteredDate = ?
GROUP BY                                             
    i.PUBLISHER_CODE
    ,i.ISBN
    ,i.SHORT_TITLE
    ORDER BY
    sum(ho.Quantity) desc

'''

In [38]:
# YTD CB / DP Sales

query10 = f'''                 
SELECT                     
    case
        when i.PUBLISHER_CODE = 'Chronicle' then 'CB'
        else 'DP'
    end div
    ,case
        when i.PUBLISHING_GROUP in('FWN','LIF') then 'FLS'
        when i.PUBLISHER_CODE = 'Levine Querido' then 'LQ'
        when i.PUBLISHER_CODE = 'Laurence King' then 'LK'
        when i.PUBLISHER_CODE = 'Creative Company' then 'CC'
        when i.PUBLISHER_CODE = 'Princeton' then 'PA'
        when i.PUBLISHER_CODE = 'Hardie Grant Publishing' then 'HG'
        when i.PUBLISHER_CODE = 'Galison' then 'GA'
        when i.PUBLISHER_CODE = 'Tourbillon' then 'TW'
        when left(i.PUBLISHING_GROUP,3) = 'BAR' then 'BAR'
        when i.PUBLISHER_CODE = 'Do Books' then 'DO'
        when i.PUBLISHER_CODE = 'Sierra Club' then 'SC'
        when i.PUBLISHER_CODE = 'AMMO Books' then 'AM'
        else i.PUBLISHING_GROUP
    end pgrp
    ,case
        when i.PRODUCT_TYPE in('BK','FT') then 'core'
        when i.PRODUCT_TYPE in('RP','CP') then 'cprp'
        else i.PRODUCT_TYPE
    end pt
    ,sum(ims.SalesNetValue) val               
FROM                       
    [summary].TitleMonthlySales ims                 
            inner join ebs.Item i on ims.ITEM_ID = i.ITEM_ID              
WHERE                      
    ims.PERIOD between '202401' AND '202412'               
    AND ims.SALETYPECODE = 'N'               
    AND i.PRODUCT_TYPE in ('BK','FT','CP','RP','DI','PZ')  
    and i.PUBLISHING_GROUP not in('MKT','PQB','GLM')
    and i.PUBLISHER_CODE not in('Moleskine','Benefit')
    and ims.DISTRIBUTION_DIRECT = 'N'
GROUP BY                          
    case
        when i.PUBLISHER_CODE = 'Chronicle' then 'CB'
        else 'DP'
    end
    ,case
        when i.PUBLISHING_GROUP in('FWN','LIF') then 'FLS'
        when i.PUBLISHER_CODE = 'Levine Querido' then 'LQ'
        when i.PUBLISHER_CODE = 'Laurence King' then 'LK'
        when i.PUBLISHER_CODE = 'Creative Company' then 'CC'
        when i.PUBLISHER_CODE = 'Princeton' then 'PA'
        when i.PUBLISHER_CODE = 'Hardie Grant Publishing' then 'HG'
        when i.PUBLISHER_CODE = 'Galison' then 'GA'
        when i.PUBLISHER_CODE = 'Tourbillon' then 'TW'
        when left(i.PUBLISHING_GROUP,3) = 'BAR' then 'BAR'
        when i.PUBLISHER_CODE = 'Do Books' then 'DO'
        when i.PUBLISHER_CODE = 'Sierra Club' then 'SC'
        when i.PUBLISHER_CODE = 'AMMO Books' then 'AM'
        else i.PUBLISHING_GROUP
    end
    ,case
        when i.PRODUCT_TYPE in('BK','FT') then 'core'
        when i.PRODUCT_TYPE in('RP','CP') then 'cprp'
        else i.PRODUCT_TYPE
    end
'''

In [39]:
# YTD CB / DP Sales but using the ebs.sales table

query10_b = f'''                 
SELECT
    case
        when i.PUBLISHER_CODE = 'Chronicle' then 'CB'
        else 'DP'
    end div
    ,case
        when i.PUBLISHING_GROUP in('FWN','LIF') then 'FLS'
        when i.PUBLISHER_CODE = 'Levine Querido' then 'LQ'
        when i.PUBLISHER_CODE = 'Laurence King' then 'LK'
        when i.PUBLISHER_CODE = 'Creative Company' then 'CC'
        when i.PUBLISHER_CODE = 'Princeton' then 'PA'
        when i.PUBLISHER_CODE = 'Hardie Grant Publishing' then 'HG'
        when i.PUBLISHER_CODE = 'Galison' then 'GA'
        when i.PUBLISHER_CODE = 'Tourbillon' then 'TW'
        when left(i.PUBLISHING_GROUP,3) = 'BAR' then 'BAR'
        when i.PUBLISHER_CODE = 'Do Books' then 'DO'
        when i.PUBLISHER_CODE = 'Sierra Club' then 'SC'
        when i.PUBLISHER_CODE = 'AMMO Books' then 'AM'
        else i.PUBLISHING_GROUP
    end pgrp
    ,case
        when i.PRODUCT_TYPE in('BK','FT') then 'core'
        when i.PRODUCT_TYPE in('RP','CP') then 'cprp'
        else i.PRODUCT_TYPE
    end pt
    ,sum(sd.REVENUE_AMOUNT) val 

FROM
    ebs.Sales sd
    inner join ssr.SalesSSRRow stie on stie.CUSTOMER_TRX_LINE_ID = sd.CUSTOMER_TRX_LINE_ID
        inner join ssr.SSRRow ssr_row on ssr_row.SSRRowID= stie.SSRRowID
        INNER JOIN ebs.Item i on i.ITEM_ID = sd.ITEM_ID

    WHERE
        Sd.PERIOD >= '202401'
        AND sd.INVOICE_LINE_TYPE='SALE'
        AND cbq2.dbo.fnSaleTypeCode(SD.AR_TRX_TYPE_ID)='N'
        AND i.PRODUCT_TYPE in ('BK','FT','CP','RP','DI','PZ')  
        and i.PUBLISHING_GROUP not in('MKT','PQB','GLM')
        and i.PUBLISHER_CODE not in('Moleskine','Benefit')
        and stie.DISTRIBUTION_DIRECT = 'N'
    GROUP BY
    case
        when i.PUBLISHER_CODE = 'Chronicle' then 'CB'
        else 'DP'
    end
    ,case
        when i.PUBLISHING_GROUP in('FWN','LIF') then 'FLS'
        when i.PUBLISHER_CODE = 'Levine Querido' then 'LQ'
        when i.PUBLISHER_CODE = 'Laurence King' then 'LK'
        when i.PUBLISHER_CODE = 'Creative Company' then 'CC'
        when i.PUBLISHER_CODE = 'Princeton' then 'PA'
        when i.PUBLISHER_CODE = 'Hardie Grant Publishing' then 'HG'
        when i.PUBLISHER_CODE = 'Galison' then 'GA'
        when i.PUBLISHER_CODE = 'Tourbillon' then 'TW'
        when left(i.PUBLISHING_GROUP,3) = 'BAR' then 'BAR'
        when i.PUBLISHER_CODE = 'Do Books' then 'DO'
        when i.PUBLISHER_CODE = 'Sierra Club' then 'SC'
        when i.PUBLISHER_CODE = 'AMMO Books' then 'AM'
        else i.PUBLISHING_GROUP
    end
    ,case
        when i.PRODUCT_TYPE in('BK','FT') then 'core'
        when i.PRODUCT_TYPE in('RP','CP') then 'cprp'
        else i.PRODUCT_TYPE
    end
'''

In [40]:
# SSR YTD Sales

query11 = f'''                 
SELECT
    ssr_row.Description ssr
    ,case
        when i.PUBLISHING_GROUP in('FWN','LIF') then 'FLS'
        when left(i.PUBLISHING_GROUP,3) = 'BAR' then 'FLS'
        when i.PUBLISHER_CODE = 'Chronicle' then i.PUBLISHING_GROUP
        when i.PUBLISHER_CODE = 'Laurence King' then 'LK'
        when i.PUBLISHER_CODE = 'Hardie Grant Publishing' then 'HG'
        when i.PUBLISHER_CODE = 'Levine Querido' then 'LQ'
        when i.PUBLISHER_CODE = 'Princeton' then 'PA'
        when i.PUBLISHER_CODE = 'Laurence King' then 'LK'
        when i.PUBLISHER_CODE = 'Sierra Club' then 'SC'
        when i.PUBLISHER_CODE = 'Creative Company' then 'CC'
        when i.PUBLISHER_CODE = 'AMMO Books' then 'AM'
        when i.PUBLISHER_CODE = 'Tourbillon' then 'TW'
        when i.PUBLISHING_GROUP = 'GAL' then 'GA'
        when i.PUBLISHING_GROUP = 'GAL' then 'GA'
        when i.PUBLISHING_GROUP = 'MUD' then 'MP'
        when i.PUBLISHING_GROUP = 'GAL-CL' then 'CL'
        when i.PUBLISHING_GROUP = 'GAL-BM' then 'BM'
        when i.PUBLISHER_CODE = 'Do Books' then 'DO'
        else i.PUBLISHING_GROUP
    end pbgrp
    ,sum(sd.REVENUE_AMOUNT) val
    ,sum(sd.QUANTITY_INVOICED) units
FROM ebs.Sales sd
    inner join ssr.SalesSSRRow stie on stie.CUSTOMER_TRX_LINE_ID = sd.CUSTOMER_TRX_LINE_ID
    inner join ssr.SSRRow ssr_row on ssr_row.SSRRowID= stie.SSRRowID
    INNER JOIN ebs.Item i on i.ITEM_ID=sd.ITEM_ID  
    inner join ssr.SubChannel subchan on subchan.SubChannelID = ssr_row.SubChannelID
    inner join ssr.Channel chan on chan.ChannelID = subchan.ChannelID
    inner join ebs.SalesRep sr on sr.SALESREP_ID = sd.PRIMARY_SALESREP_ID
WHERE
    sd.PERIOD >= '202401'
    and cbq2.dbo.fnSaleTypeCode(SD.AR_TRX_TYPE_ID)='N'   
    AND sd.INVOICE_LINE_TYPE = 'SALE'
    and i.PRODUCT_TYPE in('FT','BK')
    and i.PUBLISHER_CODE not in('Moleskine','Benefit','Glam Media','PQ Blackwell')
    and stie.DISTRIBUTION_DIRECT = 'N'
GROUP BY
    ssr_row.Description
    ,case
        when i.PUBLISHING_GROUP in('FWN','LIF') then 'FLS'
        when left(i.PUBLISHING_GROUP,3) = 'BAR' then 'FLS'
        when i.PUBLISHER_CODE = 'Chronicle' then i.PUBLISHING_GROUP
        when i.PUBLISHER_CODE = 'Laurence King' then 'LK'
        when i.PUBLISHER_CODE = 'Hardie Grant Publishing' then 'HG'
        when i.PUBLISHER_CODE = 'Levine Querido' then 'LQ'
        when i.PUBLISHER_CODE = 'Princeton' then 'PA'
        when i.PUBLISHER_CODE = 'Laurence King' then 'LK'
        when i.PUBLISHER_CODE = 'Sierra Club' then 'SC'
        when i.PUBLISHER_CODE = 'Creative Company' then 'CC'
        when i.PUBLISHER_CODE = 'AMMO Books' then 'AM'
        when i.PUBLISHER_CODE = 'Tourbillon' then 'TW'
        when i.PUBLISHING_GROUP = 'GAL' then 'GA'
        when i.PUBLISHING_GROUP = 'GAL' then 'GA'
        when i.PUBLISHING_GROUP = 'MUD' then 'MP'
        when i.PUBLISHING_GROUP = 'GAL-CL' then 'CL'
        when i.PUBLISHING_GROUP = 'GAL-BM' then 'BM'
        when i.PUBLISHER_CODE = 'Do Books' then 'DO'
        else i.PUBLISHING_GROUP
    end
'''

In [41]:
df_q1.head(10)

NameError: name 'df_q1' is not defined

In [42]:
df_q2 = pd.read_sql_query(query2,conn,params=[prior_day])

In [43]:
df_q2.head()

,ssr_row,Chronicle,Distribution,Total
0,Amazon.com,109659.39,42094.97,151754.36
1,Ingram,27286.89,32534.63,59821.52
2,American Wholesale Book Co.,13941.55,31092.05,45033.60
3,Readerlink,16860.14,17737.56,34597.70
4,Barnes & Noble,28208.26,1144.47,29352.73


In [44]:
df_q3 = pd.read_sql_query(query3,conn,params=[prior_day])

In [45]:
print(df_q3.shape)
df_q3.head()

(4003, 8)


,div,ISBN,Title,Price,pgc,Units,Dollars,order
0,cb,9781452139111,Construction Site on Christmas Night* hc,16.99,CHL,3204.0,25044.62,1
1,cb,9781452144009,Cooking Up a Storm 10th Anniversary,32.50,FLS,569.0,8693.10,2
2,cb,9780811867795,Heroes of the Environment hc,16.99,CHL,1035.0,7062.67,3
3,cb,9781452179612,From Crook to Cook hc,24.95,FLS,573.0,6708.06,4
4,cb,9781797226729,Construction Site: A Thankful Night,12.99,CHL,1116.0,6523.57,5


In [46]:
df_q3 = df_q3[df_q3['order']<11].iloc[:,1:-1]
df_q3

,ISBN,Title,Price,pgc,Units,Dollars
0,9781452139111,Construction Site on Christmas Night* hc,16.99,CHL,3204.0,25044.62
1,9781452144009,Cooking Up a Storm 10th Anniversary,32.50,FLS,569.0,8693.10
2,9780811867795,Heroes of the Environment hc,16.99,CHL,1035.0,7062.67
3,9781452179612,From Crook to Cook hc,24.95,FLS,573.0,6708.06
4,9781797226729,Construction Site: A Thankful Night,12.99,CHL,1116.0,6523.57
5,9781452174792,Canvas One Line A Day,16.95,ART,644.0,5049.73
6,9781452173634,Cat Tarot,19.95,ENT,446.0,4282.46
7,9781797221922,Construction Site: Taking Flight!,17.99,CHL,492.0,4025.79
8,5055923767108,Game Selfish Space,20.00,RID,391.0,3605.60
9,9781452164472,Richard McGuire's Playing Cards,14.95,ART,541.0,3479.16


In [47]:
df_q4 = pd.read_sql_query(query4,conn)

In [48]:
print(df_q4.shape)
df_q4.head()

(4836, 8)


,div,ISBN,Title,Price,pgc,Units,Dollars,order
0,cb,9781452139111,Construction Site on Christmas Night* hc,16.99,CHL,3205.0,25052.44,1
1,cb,9781452179612,From Crook to Cook hc,24.95,FLS,1759.0,21444.03,2
2,cb,9781452182278,LEGO Minifigure Puzzle,17.95,ENT,2479.0,20471.25,3
3,cb,5055923785225,Trivia Star Wars,13.00,RID,2112.0,12355.20,4
4,cb,9781797234137,Game Funky Fungi boxed (MTO only),12.00,RID,2280.0,12312.00,5


In [49]:
df_q4 = df_q4[df_q4['order']<11].iloc[:,1:-1]
df_q4.head()

,ISBN,Title,Price,pgc,Units,Dollars
0,9781452139111,Construction Site on Christmas Night* hc,16.99,CHL,3205.0,25052.44
1,9781452179612,From Crook to Cook hc,24.95,FLS,1759.0,21444.03
2,9781452182278,LEGO Minifigure Puzzle,17.95,ENT,2479.0,20471.25
3,5055923785225,Trivia Star Wars,13.00,RID,2112.0,12355.20
4,9781797234137,Game Funky Fungi boxed (MTO only),12.00,RID,2280.0,12312.00


In [50]:
df_q5 = pd.read_sql_query(query5,conn)

In [51]:
df_q6 = pd.read_sql_query(query6,conn,params = [prior_day])

In [52]:
df_q6

,channel,SSR_Row,qty
0,Trade,Barnes & Noble,10929
1,Export,New Zealand,2620
2,Specialty,Crate & Barrel,2520
3,Export,Asia,1705
4,Specialty,Stephen Young - Mountains,1126
5,Specialty,Harper Group (NY/NJ),1099
6,Specialty,Stephen Young,1088
7,Trade,Barnes & Noble College,715
8,Specialty,AMCI,710
9,Specialty,Kelley & Crew,686


In [53]:
df_q7 = pd.read_sql_query(query7,conn,params = [prior_day])

In [54]:
df_q8 = pd.read_sql_query(query8,conn,params = [prior_day])

In [55]:
df_q9 = pd.read_sql_query(query9,conn,params = [prior_day])

In [56]:
path = 'G:\\SALES\\2024 Sales Reports\\SSR\\SSR_Template\\rbh_daily_py.xlsx'
writer = pd.ExcelWriter(path, engine='xlsxwriter')

df_q1.to_excel(writer,sheet_name='df_q1',index = False)
df_q2.to_excel(writer,sheet_name='df_q2',index = False)
df_q3.to_excel(writer,sheet_name='df_q3',index = False)
df_q4.to_excel(writer,sheet_name='df_q4',index=False)
df_q5.to_excel(writer,sheet_name='df_q5',index=False)
df_q6.to_excel(writer,sheet_name='df_q6',index=False)
df_q7.to_excel(writer,sheet_name='df_q7',index=False)
df_q8.to_excel(writer,sheet_name='df_q8',index=False)
df_q9.to_excel(writer,sheet_name='df_q9',index=False)

writer.save()

NameError: name 'df_q1' is not defined

In [ ]:
# Calculating the total of the Chronicle column
chronicle_total = df_q2['Chronicle'].sum()

# Formatting the result
result = f"Chronicle Prior-Day Total: ${chronicle_total:,.2f}"

print(result)

In [ ]:
end = dt.now()
print(end)

In [ ]:
total_running_time = end - start
print(total_running_time)